In [2]:
from selenium import webdriver

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchFrameException

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup as bs

from time import sleep

from tqdm import tqdm

from webdriver_manager.chrome import ChromeDriverManager

import csv




#driver.find_element_by_xpath('//*[@id="searchResults"]/div[2]/span').click()


In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://rg.ru/doc-search/'
ssilka = []
host = 'https://rg.ru/'
driver.get(url)
driver.maximize_window()


#sleep(20)
#Wait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="rgb_feed_doc-binding"]/div/div/div/div/a'))).click()

In [ ]:
def search_more():
    try:
        sleep(10)
        element = Wait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchResults"]/div[2]/span')))
        element.click()
        print('success')
    except ElementClickInterceptedException:
        sleep(5)
        action = ActionChains(driver)
        action.key_down(Keys.END).perform()
        sleep(5)
        print('performing keys...')
        element = Wait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchResults"]/div[2]/span')))
        element.click()
    except:
        print("cannot search more")
        

for i in range(5):
    action = ActionChains(driver)
    action.key_down(Keys.END).perform()
    search_more()
print('good')

In [ ]:
pages = []

soup = bs(driver.page_source, 'html.parser')

with open('articles_links.txt', 'w') as file:
    for article in soup.findAll('div', class_='b-news-inner__list-item-wrapper'):
        link = article.find('a', href=True)
        wholelink = host+link['href']
        file.write(wholelink)
        file.write('\n')
        page = link['href']
        print(wholelink)
        pages.append(page)

        
with open('pages.txt', 'w') as page:
    page.write(str(pages))

In [ ]:
print(len(pages))

In [ ]:

#Checks if Oops window opens
def oops_factor():
    ops = soup1.find('body')
    if ops != None:
        ops_text = ops.get_text()
        if 'oops' in ops_text.lower():
            driver.refresh()
            sleep(3)

In [ ]:
import csv


def accident(soup1, driver):
    sleep(20)
    material = soup1.find('div', class_='b-sticked-material-share b-sticked-material-share__header')
    if material != None:
        return material.get_text()
    else:
        material = 'N/A'
        return material

In [ ]:
def getting_text(driver, soup1):
    try:
        sleep(3)
        material = driver.find_element_by_xpath('//*[@id="rgb_material-wrapper_doc"]/article')
        if material is not None: 
            return material.text
        else:
            material = 'N/A'
            return material
    except AttributeError:
        sleep(3)
        material = soup1.find('article', attrs={'class': 'b-material-wrapper__body'})
        if material is None:
            return accident(soup1, driver)
            # аварийный вариант    
        else:
            print('Text for', driver.current_url)
            return material.get_text()
        material = 'N/A'
        return material
    except:
        print('NOT WORKING', driver.current_url)
        material = 'N/A'
        return material

In [ ]:
def getting_comment(driver):
    sleep(5)
    try:
        comment = driver.find_element_by_xpath('//*[@id="rgb_feed_doc-binding"]/div/div[2]/div/div/a')
        if comment is not None:
            if 'Комментарии Российской Газеты' in comment.text:
                comment.click()
                comment_text = driver.find_element_by_xpath('//*[@id="articleContainer"]/div[1]')
                sleep(5)
                if comment_text is not None:
                    return comment_text.text
                else:
                    comment_text = 'N/A'
                    return comment_text
            else:
                comment_text = 'N/A'
                return comment_text
        else:
            comment_text = 'N/A'
            return comment_text
    except NoSuchElementException:
        comment_text= 'N/A'
        return comment_text

In [ ]:
#('a', 'b-link b-link_title')
    
# Это я намудрил    
def appropriate(soup1, driver):
    test_num = 0
    sleep(15)
    comment = driver.find_element_by_xpath('//*[@id="rgb_feed_doc-binding"]/div/div[2]/div/div/a')
    if comment is not None:
        if 'Комментарии Российской Газеты' in comment.text:
            sleep(3)
            test_num += 2
            material = driver.find_element_by_xpath('//*[@id="rgb_material-wrapper_doc"]/article')
            if material is not None:   
                test_num += 2
            else:
                accident(soup1, driver)
                if material is not None:
                    test_num += 2
                else:
                    test_num -= 2
    else: 
        test_num -= 2
    return test_num

In [ ]:
def document_date(soup1):
    sleep(3)
    date = driver.find_element_by_xpath('//*[@id="rgb_material-head_art"]/div[2]/div[1]/span[1]')
    if date is not None:
        return date.text
    else:
        date = 'N/A'
        return date

In [1]:
def saving():
    with open('mydataset.csv', 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(['Название документа', 'Ссылка', 'Текст', 'Комментарий РГ', 'Дата'])
        return writer
        

def data_making():
    # NOT READY!!!!
    table.append(
        {
            'Название документа': title,
            'Ссылка': driver.current_url, 
            'Текст': getting_text(driver, soup1, link),
            'Комментарий РГ': getting_comment(soup1, host),
            'Дата': document_date(soup1),
        }
    )
    saving().writerow([title, driver.current_url, getting_text(driver, soup1, link), getting_comment(soup1, host), document_date(soup1),])
    return table



    
    
# Нужно сделать так, чтобы программа проверяла на наличие комментария, если его нет - статью мимо
n = 0
link_comment = []
link_comment_reserve = []
table = []
with open('links_for_comments.txt', 'w', encoding='utf-8') as comment_links:          
    for link in pages:
        sleep(2)
        try:
            driver.get(host+link)
        except:
            driver.navigate(link)
            print('Problem with link for', link)
        sleep(10)
        
            
        soup1 = bs(driver.current_url, 'html.parser')
        title = driver.title
        driver.implicitly_wait(10)

        
        oops_factor()
        
        if appropriate(soup1, driver) > 3:
            sleep(5)
            data_making()
            n += 1
            print('Successful for ', n)
        print(f'Unable to parse {driver.current_url}    Sorry!')    
        
print('I finished')
            
# material = [x.get_text() for x in soup1.find('article', attrs={'class': 'b-material-wrapper__body'})]    
# zeug = [x.get_text() for x in soup.find_all('div', attrs={'class': 'fm_linkeSpalte'})]
# #rgb_material-wrapper_doc > article   

NameError: name 'pages' is not defined